In [1]:
import os
import sys

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import librosa
import librosa.display

import tensorflow as tf
from keras.models import load_model

import tensorflow as tf 
import keras.backend as K 
# K.tensorflow_backend.set_session(tf.Session(config=tf.ConfigProto(device_count = {'GPU': 0}))) 


Using TensorFlow backend.


In [2]:
threshold = 0.5
dt = 30./1500.

def make_result_plot(wav, predicted, gt_df, prd_df, fig_title='sample', save_fig=False, filename='sample.png'):
    a, sr = librosa.load(wav)
    # mel spectrogram
    mel = librosa.feature.melspectrogram(y=a, sr=sr)
    mel_dB = librosa.power_to_db(mel, ref=np.max)

    # ===============================
    gt_flag_start = gt_df['onset']
    gt_flag_end   = gt_df['offset']

    # ===============================
    time_index = predicted.shape[0]
    left = np.array(range(time_index))*dt

    # ===============================

    plt.rcParams["font.size"] = 14
    fig = plt.figure(figsize=(14, 10))

    ax1 = fig.add_subplot(411)
    disp = librosa.display.specshow(mel_dB, x_axis='time', y_axis='mel', sr=sr)
    plt.title('Mel-frequency spectrogram')
    cbaxes = fig.add_axes([0.9, 0.735, 0.01, 0.17]) 
    cb   = plt.colorbar(format='%+2.0f dB', cax=cbaxes)

    ax2 = fig.add_subplot(412)
    plt.bar(left, predicted, width=dt)
    plt.plot([left[0], left[-1]],[0.5, 0.5], "red", linestyle='dashed') # normal way
    plt.xlim(left[0], left[-1])
    plt.ylim([0, 1])
    plt.title('Probability')

    ax3 = fig.add_subplot(413)
    plt.tick_params(left=False)
    plt.yticks(color="None")
    plt.title('Predicted')

    for index, row in prd_df.iterrows():
        flag_start = row[1]
        flag_end   = row[2]    
        testx = [flag_start, flag_end, flag_end, flag_start]
        testy = [0,0,1,1]
        plt.xlim([0, 30])
        plt.ylim([0, 1])
        plt.fill(testx,testy,color="blue")


    ax4 = fig.add_subplot(414)
    plt.tick_params(left=False)
    plt.yticks(color="None")
    plt.title('Ground Truth')
    testx = [gt_flag_start, gt_flag_end, gt_flag_end, gt_flag_start]
    testy = [0,0,1,1]
    plt.xlim([0, 30])
    plt.ylim([0, 1])
    plt.fill(testx,testy,color="red")

    fig.tight_layout()

    #幅をax2と同じにする
    fig.canvas.draw()
    axpos1 = ax1.get_position() # 上の図の描画領域
    axpos2 = ax2.get_position() # 下の図の描画領域
    axpos3 = ax3.get_position() # 下の図の描画領域
    axpos4 = ax4.get_position() # 下の図の描画領域
    width = axpos1.width - 0.1
    ax1.set_position([axpos1.x0, axpos1.y0 - 0.05, width, axpos1.height])
    ax2.set_position([axpos1.x0, axpos2.y0 - 0.05, width, axpos2.height])
    ax3.set_position([axpos1.x0, 0.35, width, 0.03])
    ax4.set_position([axpos1.x0, 0.23, width, 0.03])

    fig.suptitle(fig_title, fontsize=20)
    if save_fig:
        plt.savefig(filename)
    else:
        plt.show()

    plt.close(fig)


In [3]:
env = os.environ

baseline_dir = env['DCASE_BASE']
work_dir     = env['MYWORK_BASE']

sys.path.append(baseline_dir)
sys.path.append(work_dir)

from main.src.support.analyzer import CakirAnalyzer

base_dir     = work_dir + '/data/TUT-rare-sound-events-2017-development/'
feature_dir  = work_dir + '/main/results/my_sample/feature_extractor/dataset_c17f65145b5c15a1a0afc148dc881423/feature_extractor_00ba1657b4edcc51c91ce0d45910ebb5/'

col_names = ['filename', 'onset', 'offset', 'label']
# labels = ['babycry', 'gunshot', 'glassbreak']
labels = ['glassbreak']

project_name= 'my_sample'
set_id = 'crnn'


# Cakir Table1. units 96 --> 'eval_[5aa3cfc96be6061a47013f0c05ffeb3b].yaml'
# model_dir = work_dir + 'main/results/my_sample/learner/dataset_c17f65145b5c15a1a0afc148dc881423/feature_extractor_9138a38a27a1fe93ce9267c74b646c84/feature_normalizer_86c6764b339414c628793552d084ff29/feature_aggregator_baaf606d9ac1eaca43a6a24b599998a9/learner_35b914298a367b0dddb4dca085381e67/'
# recog_dir = work_dir + 'main/results/my_sample/recognizer/dataset_c17f65145b5c15a1a0afc148dc881423/feature_extractor_9138a38a27a1fe93ce9267c74b646c84/feature_normalizer_86c6764b339414c628793552d084ff29/feature_aggregator_baaf606d9ac1eaca43a6a24b599998a9/learner_35b914298a367b0dddb4dca085381e67/recognizer_c5143b74ef67d78e7a154ca3808faf4b/'
# plot_basedir = '../../plots/predict_comparison/mixtures_devtest_0367e094f3f5c81ef017d128ebff4a3c/Cakir_T1U96/'

# Cakir Table1. units 32 -->'eval_[ed4beb6d218c011b1888395c4b5f2aad].yaml'
# model_dir = work_dir + 'main/results/my_sample/learner/dataset_c17f65145b5c15a1a0afc148dc881423/feature_extractor_9138a38a27a1fe93ce9267c74b646c84/feature_normalizer_86c6764b339414c628793552d084ff29/feature_aggregator_baaf606d9ac1eaca43a6a24b599998a9/learner_dc2acc4ef527864e2910eeeb8896a043/'
# recog_dir = work_dir + 'main/results/my_sample/recognizer/dataset_c17f65145b5c15a1a0afc148dc881423/feature_extractor_9138a38a27a1fe93ce9267c74b646c84/feature_normalizer_86c6764b339414c628793552d084ff29/feature_aggregator_baaf606d9ac1eaca43a6a24b599998a9/learner_dc2acc4ef527864e2910eeeb8896a043/recognizer_c5143b74ef67d78e7a154ca3808faf4b/'
# plot_basedir = '../../plots/predict_comparison/mixtures_devtest_0367e094f3f5c81ef017d128ebff4a3c/Cakir_T1U32/'

# Cakir Table1. units 160 --> 'eval_[d96bdb1e551eba73fb75755375d7af3b].yaml'
model_dir = work_dir + 'main/results/my_sample/learner/dataset_c17f65145b5c15a1a0afc148dc881423/feature_extractor_9138a38a27a1fe93ce9267c74b646c84/feature_normalizer_86c6764b339414c628793552d084ff29/feature_aggregator_baaf606d9ac1eaca43a6a24b599998a9/learner_419c1ef3fe8202709628e692f51ac957/'
recog_dir = work_dir + 'main/results/my_sample/recognizer/dataset_c17f65145b5c15a1a0afc148dc881423/feature_extractor_9138a38a27a1fe93ce9267c74b646c84/feature_normalizer_86c6764b339414c628793552d084ff29/feature_aggregator_baaf606d9ac1eaca43a6a24b599998a9/learner_419c1ef3fe8202709628e692f51ac957/recognizer_c5143b74ef67d78e7a154ca3808faf4b/'
plot_basedir = '../../plots/predict_comparison/mixtures_devtest_0367e094f3f5c81ef017d128ebff4a3c/Cakir_T1U160/'

#=================================================
### test
# meta_babycry_prd = recog_dir + 'results_fold1_babycry.txt'
# meta_babycry_gt  = base_dir + 'generated_data/mixtures_devtest_0367e094f3f5c81ef017d128ebff4a3c/meta/event_list_devtest_babycry.csv'

# prd_df_bc = pd.read_csv(meta_babycry_prd, sep='\t', header=None, names=col_names)
# gt_df_bc  = pd.read_csv(meta_babycry_gt, sep='\t', header=None, names=col_names)

# prd_dfs = {'babycry': df_bc, 'gunshot': df_gs, 'glassbreak': df_gb}
# gt_dfs = {'babycry': df_bc, 'gunshot': df_gs, 'glassbreak': df_gb}
# model_bc_fn = model_dir + 'model_fold1_babycry.model.hdf5'
# model_bc_fn = model_dir + 'model_fold1_babycry'




In [4]:
# make plots

for label in labels:
    plot_dir = plot_basedir + label + '/'
    if not(os.path.exists(plot_dir)):
        os.makedirs(plot_dir)    
    
    meta_prd = recog_dir + 'results_fold1_' + label + '.txt'
    meta_gt  = base_dir + 'generated_data/mixtures_devtest_0367e094f3f5c81ef017d128ebff4a3c/meta/event_list_devtest_' + label + '.csv'    
    prd_df = pd.read_csv(meta_prd, sep='\t', header=None, names=col_names)
    gt_df  = pd.read_csv(meta_gt, sep='\t', header=None, names=col_names)
    
    model_fn = model_dir + 'model_fold1_' + label
    ca = CakirAnalyzer(model_fn, project_name, set_id)

    for index, row in gt_df.iterrows():
        wav_name = row[0]
        wav = base_dir + wav_name

        feature_filename = feature_dir + 'sequence_' + wav_name.split('/')[-1].split('.')[0] + '.cpickle'
        predicted = ca.predict(feature_filename)[0]

        this_gt_df = row
        this_prd_df = prd_df[prd_df.filename == wav_name]
        
        title = label + '_test_' + str(index)
        plot_finename = plot_dir + os.path.split(wav)[1].split('.')[0] + '.png'
        
#         make_result_plot(wav, predicted, this_gt_df, this_prd_df)
        make_result_plot(wav, predicted, this_gt_df, this_prd_df, fig_title=title, save_fig=True, filename=plot_finename)

/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/dcase_framework/files.py:379: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dict.update(self, yaml.load(infile))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/home/hirotoshi/.pyenv/versions/anaconda3-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:61: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [19]:
feature_dir      = work_dir + '/main/results/my_sample/feature_extractor/dataset_c17f65145b5c15a1a0afc148dc881423/feature_extractor_00ba1657b4edcc51c91ce0d45910ebb5/'
feature_filename = feature_dir + 'sequence_' + 'mixture_devtest_babycry_000_c02f92b79f2bbefa98d008f3c2d9b704.cpickle'

feature_filename

'/store/projects/ml/mathG/DCASE2017/myWork//main/results/my_sample/feature_extractor/dataset_c17f65145b5c15a1a0afc148dc881423/feature_extractor_00ba1657b4edcc51c91ce0d45910ebb5/sequence_mixture_devtest_babycry_000_c02f92b79f2bbefa98d008f3c2d9b704.cpickle'

In [26]:
a = meta_prd_df['filename'][0]
a.split('/')[-1].split('.')[0]

'mixture_devtest_babycry_000_c02f92b79f2bbefa98d008f3c2d9b704'